In [20]:
import tensorflow.compat.v2.feature_column as fc  

import tensorflow as tf
import pandas as pd

import matplotlib.pyplot as plt 
from IPython.display import *
from six.moves import urllib

from tabulate import tabulate

In [21]:
dftrain = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
dfeval = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

CATEGORICAL_COLUMN = ['sex','n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
NUMERIC_COLUMN = ['age', 'fare']

feature_column = []

for feature_name in CATEGORICAL_COLUMN:
    vocabulary = dftrain[feature_name].unique() # gets a list of all unique values in column 
    feature_column.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
    # create a feature column which takes as input some string and outputs a one-hot vector

for feature_name in NUMERIC_COLUMN:
    feature_column.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_column)
clear_output()

In [22]:
dftrain["embark_town"].unique()

array(['Southampton', 'Cherbourg', 'Queenstown', 'unknown'], dtype=object)

## Input function

With linear regression

In [23]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():
        # create tf.data.dataset.from_tensor_slices object with data and its label
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)  
        return ds
    return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle = False)

linear_est = tf.estimator.LinearClassifier(feature_columns = feature_column)

linear_est.train(train_input_fn)

result = linear_est.evaluate(eval_input_fn)

clear_output()

In [24]:
print(result['accuracy'])

0.77272725


In [25]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[3])
print(result[3]['probabilities'][1])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmpyx3sma6l\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                        female
age                          55.0
n_siblings_spouses              0
parch                           0
fare                         16.0
class                      Second
deck                      unknown
embark_town           Southampton
alone                           y
Name: 3, dtype: object
0.6616521


### Dataset

This specific dataset separates data (here is flowers) into 3 different classes of species:
- Setosa 
- Versicolor
- Virginica <br>

The information abour each flower is following:
- sepal length
- sepal width
- petal length
- petal width


In [26]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Define some constants to help us later on

In [27]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
)
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
)

train = pd.read_csv(train_path, names = CSV_COLUMN_NAMES, header = 0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header = 0)

In [28]:
# Look at our data
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


We can pop the species column off and use that as our label

In [29]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [30]:
train.shape # we have 120 entries with 4 features

(120, 4)

In [31]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

#### We have input function for this dataset

In [32]:
def input_fn(features, labels, training=True, batch_size=256):
    #Convert the inputs to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    #Shuffle and repeat if we are in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()  

    return dataset.batch(batch_size) 


##### Feature colums

In [33]:
my_feature_columns = []
# train.keys gives all the columns 
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


## Building the model 
And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from. <br>
Some options are listed below: 
- DNNClassifier (Deep Neural Network)
- LinearClassifier

We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear correspondence in out data. 

In [34]:
# Build a DNN with a hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively
    hidden_units=[30,10],
    n_classes = 3
)
clear_output()

#### Training

In [35]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps = 5000 
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\ADMIN\AppData\Local\Temp\tmpn4c2hwy4\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.087018, step = 0
INFO:tensorflow:global_step/sec: 509.246
INFO:tensorflow:loss = 0.90413284, step = 100 (0.196 sec)
INFO:tensorflow:global_step/sec: 757.614
INFO:tensorflow:loss = 0.8541671, step = 200 (0.132 sec)
INFO:tensorflow:global_step/sec: 885.723
INFO:tensorflow:loss = 0.81157666, step = 300 (0.113 sec)
INFO:tensorflow:global_step/sec: 623.853
INFO:tensorflow:loss = 0.78875697, step = 400 (0.160 sec)
INFO:tensorflow:global_step/sec: 1036.49
INFO:tensorflow:loss = 0.7534907

#### Evaluation

In [36]:
classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False)
)
eval_result= classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False)
)
print("\n Test set accuracy: {accuracy:0.3f}\n".format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-10-25T06:22:58
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmpn4c2hwy4\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.40436s
INFO:tensorflow:Finished evaluation at 2022-10-25-06:22:58
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7, average_loss = 0.5200188, global_step = 5000, loss = 0.5200188
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\ADMIN\AppData\Local\Temp\tmpn4c2hwy4\model.ckpt-5000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-10-25T06:22:59
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmpn4c2hwy4\model.ckpt-5000
INFO:ten

#### Predictions

In [37]:
def input_fn(features, batch_size=256):
    #Convert the inputs to a Dataset withour labels. 
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predict = {}
features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']

print("Please type numeric values as prompted.")
for feature in features:
    valid = True
    while valid:
        val = input(feature + ": ")  
        if not val.isdigit(): valid = False
    
    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))

for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id] 

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))
    



Please type numeric values as prompted.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADMIN\AppData\Local\Temp\tmpn4c2hwy4\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (91.5%)
